In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [2]:
train_model = r"C:\Users\1\Desktop\train_model_output\Qwen2.5-0.5B\SFT_output"

model = AutoModelForCausalLM.from_pretrained(train_model)
model = model.to(device)
tokenizer = AutoTokenizer.from_pretrained(train_model, padding_side="right")

In [3]:
tokenizer.add_special_tokens({"pad_token": "[PAD]"})

0

In [4]:
import json

with open("data.json", "r") as f:
    data =json.load(f)
data

[{'instruct': '请你给哪吒写一首诗：',
  'input': '哪吒降世，意气飞扬。\n逆天改命，破障冲霄。',
  'label': '红绫缠腕，风火踏浪。\n不屈不悔，笑傲苍茫。'},
 {'instruct': '请你给敖丙写一首诗：', 'input': '碧海生龙子，云中舞雪霜。', 'label': '恩仇难两忘，何处是家乡？'},
 {'instruct': '请你给殷夫人写一首诗：',
  'input': '十月怀胎盼子生，柔心铁骨两相承。',
  'label': '甘将慈爱护天地，不惧风雷不惧征。'},
 {'instruct': '请你给太乙真人写一首诗：', 'input': '仙风道骨，骑兽遨游。', 'label': '炉中炼术，指点神童。'},
 {'instruct': '请你给申公豹写一首诗：',
  'input': '阴谋藏心，步步为营。\n狂傲不羁，志向高冥。',
  'label': '欲翻天命，终难遂行。\n困局自招，悔恨难平。'}]

In [14]:
def infer(text):
    input_ids = tokenizer(text, return_tensors="pt").to(model.device)

    generated_ids = model.generate(**input_ids)
    generated_ids = [
        output_ids[len(input_ids) :]
        for input_ids, output_ids in zip(input_ids.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

In [18]:
print("=" * 50 + "instruct" + "=" * 50)
for item in data:
    # instruct + input -> label
    instruct, input, label = item["instruct"], item["input"], item["label"]
    print(f"text_input: {instruct + input}")
    print(f"predict: {infer(instruct + input)}")
    print(f"label: {label}")
    print("-" * 101)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


==================================================instruct==================================================
text_input: 请你给哪吒写一首诗：哪吒降世，意气飞扬。
逆天改命，破障冲霄。


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


predict: 红绫缠腕，风火踏浪。
不屈不悔，笑傲苍茫。
label: 红绫缠腕，风火踏浪。
不屈不悔，笑傲苍茫。
-----------------------------------------------------------------------------------------------------
text_input: 请你给敖丙写一首诗：碧海生龙子，云中舞雪霜。
predict: 恩仇难两忘，何处是家乡？
label: 恩仇难两忘，何处是家乡？
-----------------------------------------------------------------------------------------------------
text_input: 请你给殷夫人写一首诗：十月怀胎盼子生，柔心铁骨两相承。


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


predict: 甘将慈爱护天地，不惧风雷不惧征。
label: 甘将慈爱护天地，不惧风雷不惧征。
-----------------------------------------------------------------------------------------------------
text_input: 请你给太乙真人写一首诗：仙风道骨，骑兽遨游。
predict: 炉中炼术，指点神童。
label: 炉中炼术，指点神童。
-----------------------------------------------------------------------------------------------------
text_input: 请你给申公豹写一首诗：阴谋藏心，步步为营。
狂傲不羁，志向高冥。
predict: 欲翻天命，终难遂行。
困局自招，悔恨难平。
label: 欲翻天命，终难遂行。
困局自招，悔恨难平。
-----------------------------------------------------------------------------------------------------


In [20]:
print("=" * 50 + "instruct" + "=" * 50)
for item in data:
    # instruct + input -> label
    instruct, input, label = item["instruct"], item["input"], item["label"]
    print(f"text_input: {instruct }")
    print(f"predict: {infer(instruct)}")
    print(f"label: {label}")
    print("-" * 101)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


==================================================instruct==================================================
text_input: 请你给哪吒写一首诗：


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


predict: 红绫缠腕，风火踏浪。不屈不悔，笑傲苍茫。
label: 红绫缠腕，风火踏浪。
不屈不悔，笑傲苍茫。
-----------------------------------------------------------------------------------------------------
text_input: 请你给敖丙写一首诗：
predict: 恩仇难两忘，何处是家乡？
label: 恩仇难两忘，何处是家乡？
-----------------------------------------------------------------------------------------------------
text_input: 请你给殷夫人写一首诗：


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


predict: 恩仇难两忘，何处是家乡？
label: 甘将慈爱护天地，不惧风雷不惧征。
-----------------------------------------------------------------------------------------------------
text_input: 请你给太乙真人写一首诗：
predict: 炉中炼术，指点神童。
label: 炉中炼术，指点神童。
-----------------------------------------------------------------------------------------------------
text_input: 请你给申公豹写一首诗：
predict: 恩仇难两忘，何处是家乡？
label: 欲翻天命，终难遂行。
困局自招，悔恨难平。
-----------------------------------------------------------------------------------------------------
